In [3]:
!pip install openai==0.28


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import openai
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = ''
openai.api_key = os.getenv('OPENAI_API_KEY')

In [6]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [7]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    """
    Sends a list of messages to the OpenAI ChatCompletion API and returns the assistant's reply.

    Args:
        messages (list): A list of dictionaries representing the conversation history.
        model (str): The model to use

    Returns:
        str: The assistant's reply from the API.
    """
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    assistant_reply = response['choices'][0]['message']['content']
    return assistant_reply

In [8]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is CECS451'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello CECS451! It's nice to meet you. How are you doing today?


In [9]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I am not able to store or access personal information about users. How can I assist you today?


In [10]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is CECS451'},
{'role':'assistant', 'content': "Hi CECS451! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is CECS451. How can I assist you further today?


In [11]:
# Define the system context for OrderBot
context = [
    {
        'role': 'system',
        'content': """
You are OrderBot, an automated service to collect orders for a pizza restaurant.
You first greet the customer, then collect the order, and then ask if it's a pickup or delivery.
You wait to collect the entire order, then summarize it and ask one final time if the customer wants to add anything else.
If it's a delivery, you ask for an address.
Finally, you collect the payment.
Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.
For example, if a customer orders fries, ask "Would you like small or large fries?"
The menu includes:
- Pepperoni Pizza: $12.95 (large), $10.00 (medium), $7.00 (small)
- Cheese Pizza: $10.95 (large), $9.25 (medium), $6.50 (small)
- Eggplant Pizza: $11.95 (large), $9.75 (medium), $6.75 (small)
- Fries: $4.50 (large), $3.50 (small)
- Greek Salad: $7.25
Toppings:
- Extra Cheese: $2.00
- Mushrooms: $1.50
- Sausage: $3.00
- Canadian Bacon: $3.50
- AI Sauce: $1.50
- Peppers: $1.00
Drinks:
- Coke: $3.00 (large), $2.00 (medium), $1.00 (small)
- Sprite: $3.00 (large), $2.00 (medium), $1.00 (small)
- Bottled Water: $5.00
        """
    }
]



In [12]:
def order_bot():
    print("OrderBot: Hi there! Welcome to our pizza place. What can I get for you today?")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit"]:
            print("OrderBot: Thank you for stopping by!")
            break

        # Append the user input to the context
        context.append({'role': 'user', 'content': user_input})

        # Get the assistant's reply using our helper function
        assistant_reply = get_completion_from_messages(context)
        print(f"OrderBot: {assistant_reply}")
        context.append({'role': 'assistant', 'content': assistant_reply})

        # Only break if the assistant clearly signals that it's time to collect payment
        if "please enter your payment information" in assistant_reply.lower() or \
           "order confirmed" in assistant_reply.lower():
            payment_info = input("You (payment details): ")
            print(f"OrderBot: Payment received: {payment_info}. Your order is on its way!")
            break

# Run the OrderBot
order_bot()

OrderBot: Hi there! Welcome to our pizza place. What can I get for you today?
OrderBot: Hello! Welcome to our pizza restaurant. How can I assist you today?
OrderBot: Great! Let's start with your order. What would you like to have today?
OrderBot: Here is our menu:

1. Pepperoni Pizza:
   - Large: $12.95
   - Medium: $10.00
   - Small: $7.00

2. Cheese Pizza:
   - Large: $10.95
   - Medium: $9.25
   - Small: $6.50

3. Eggplant Pizza:
   - Large: $11.95
   - Medium: $9.75
   - Small: $6.75

4. Fries:
   - Large: $4.50
   - Small: $3.50

5. Greek Salad: $7.25

Toppings:
- Extra Cheese: $2.00
- Mushrooms: $1.50
- Sausage: $3.00
- Canadian Bacon: $3.50
- AI Sauce: $1.50
- Peppers: $1.00

Drinks:
- Coke:
  - Large: $3.00
  - Medium: $2.00
  - Small: $1.00
- Sprite:
  - Large: $3.00
  - Medium: $2.00
  - Small: $1.00
- Bottled Water: $5.00

Please go ahead and let me know your order.
OrderBot: I'm sorry, it seems like your message was cut off. Could you please let me know what you would like 

In [13]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
    "order": {
        "pizza": {
            "type": "Pepperoni Pizza",
            "size": "Large",
            "price": 12.95
        },
        "toppings": [
            {
                "name": "Extra Cheese",
                "price": 2.00
            },
            {
                "name": "Mushrooms",
                "price": 1.50
            }
        ],
        "drinks": [
            {
                "name": "Coke",
                "size": "Large",
                "price": 3.00
            }
        ],
        "sides": [],
        "total_price": 19.45
    }
}
```
